In [ ]:
from google.colab import files
files.upload()  # Select the 'kaggle.json' file from your system
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d rishitjavia/netflix-movie-rating-dataset
!unzip netflix-movie-rating-dataset.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/rishitjavia/netflix-movie-rating-dataset
License(s): CC0-1.0
 98% 73.0M/74.7M [00:00<00:00, 105MB/s]
100% 74.7M/74.7M [00:00<00:00, 112MB/s]
Archive:  netflix-movie-rating-dataset.zip
  inflating: Netflix_Dataset_Movie.csv  
  inflating: Netflix_Dataset_Rating.csv  


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

class NeuralCF:
    def __init__(self, num_users, num_items, embedding_dim=10, hidden_layers=[64, 32], activation='relu', learning_rate=0.001):
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.learning_rate = learning_rate
    def _build_model(self):
        user_input = Input(shape=(1,))
        item_input = Input(shape=(1,))
        user_embedding = Embedding(self.num_users, self.embedding_dim)(user_input)
        user_embedding = Flatten()(user_embedding)
        item_embedding = Embedding(self.num_items, self.embedding_dim)(item_input)
        item_embedding = Flatten()(item_embedding)
        vector = Concatenate()([user_embedding, item_embedding])
        for units in self.hidden_layers:
            vector = Dense(units, activation=self.activation)(vector)
        output = Dense(1, activation='sigmoid')(vector)
        model = Model(inputs=[user_input, item_input], outputs=output)
        return model
    def train(self, X_train, y_train, epochs=10, batch_size=10, validation_split=0.1):
        X_train = [X_train[:, 0], X_train[:, 1]]
        y_train = np.array(y_train)
        model = self._build_model()
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        self.model = model
    def predict(self, X_test):
        X_test = [X_test[:, 0], X_test[:, 1]]
        return self.model.predict(X_test)
# Hyperparameters that could be tuned
embedding_dim = 10
hidden_layers = [64, 32]
activation = 'relu'
learning_rate = 0.001
# The netflix rating dataset found here : https://www.kaggle.com/datasets/rishitjavia/netflix-movie-rating-dataset?select=Netflix_Dataset_Rating.csv
df = pd.read_csv("/content/Netflix_Dataset_Rating.csv").iloc[:1000,:]
X = df[['User_ID','Movie_ID']].to_numpy()
y = df['Rating'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
num_users = df['User_ID'].max() + 1
num_items = df['Movie_ID'].max() + 1
# Train and predict using NeuralCF
ncf = NeuralCF(num_users, num_items, embedding_dim, hidden_layers, activation, learning_rate)
ncf.train(X_train, y_train)
y_pred = ncf.predict(X_test)
# Evaluate using Mean Squared Error
mse = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mse)

Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 34s 427ms/step - loss: 9.8805 - val_loss: 7.8286
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 40s 421ms/step - loss: 7.9380 - val_loss: 7.7328
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 30s 423ms/step - loss: 7.8263 - val_loss: 7.7284
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 41s 419ms/step - loss: 8.1653 - val_loss: 7.7269
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 45s 481ms/step - loss: 7.7785 - val_loss: 7.7262
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 37s 425ms/step - loss: 8.0369 - val_loss: 7.7259
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 41s 431ms/step - loss: 7.9490 - val_loss: 7.7256
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 31s 425ms/step - loss: 8.0110 - val_loss: 7.7255
Epoch 9/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 43s 454ms/step - loss: 8.0068 - val_loss: 7.7254
Epoch 10/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 33s 464ms/step - loss: 7.7110 - val_loss: 7.7253
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Mean Absolute Error: 2.475059670805931
